In [1]:
import pandas as pd 

# 25번 기준 코드이니 
# 노선별로 잘 변형해서 써야함

# 좌표와 정류장, 정류장 ID가 매칭된 샘플( == row 한줄)들로  꼭 구성되어야 합니다! 

df_좌표 = pd.read_excel("정류장_좌표 (1).xlsx")
df_좌표 = df_좌표.drop('Unnamed: 0',axis= 1 )

df_좌표.head()

,정류장_ID,정류장,x,y
0,00_오이도차고지,오이도차고지,126.696250,37.341600
1,01_삼양사,삼양사,126.702683,37.334133
2,02_오이도해양단지.옥터초교입구,오이도해양단지.옥터초교입구,126.693283,37.347983
3,03_오이도중앙로입구,오이도중앙로입구,126.690683,37.347767
4,04_오이도종합어시장,오이도종합어시장,126.689667,37.345000


In [2]:
df_좌표 = df_좌표[df_좌표['정류장_ID'] != "01_삼양사"]

In [3]:
df_상행_좌표 = df_좌표.iloc[:25,:]
df_하행_좌표 = df_좌표.iloc[25:, :]

In [4]:
origin_longitude = df_좌표['x'].tolist()
origin_latitude = df_좌표['y'].tolist()
dest_longitude = df_좌표['x'].tolist()
dest_latitude = df_좌표['y'].tolist()
station_list = df_좌표['정류장_ID'].tolist()


# 주의 
# origin_dict : 상행 
# dest_dict : 하행 
origin_dict = dict(zip(station_list, zip(origin_longitude, origin_latitude)))
dest_dict = dict(zip(station_list, zip(dest_longitude, dest_latitude)))

In [5]:
import requests
import itertools
import time  # API 요청 간 텀을 줄 수 있음
from geopy import distance

# 거리 계산용 함수
def get_directions(api_key, origin, destination):
    url = "https://apis-navi.kakaomobility.com/v1/directions"

    params = {
        "origin": f"{origin[0]},{origin[1]}",
        "destination": f"{destination[0]},{destination[1]}",
        "priority": "DISTANCE"
    }

    headers = {
        "Authorization": f"KakaoAK {api_key}",
        "Content-Type": "application/json"
    }

    response = requests.get(url, headers=headers, params=params)

    if response.status_code == 200:
        return response.json()
    else:
        print(f"API 요청 실패: {response.status_code}, {response.text}")
        return None

In [6]:
def use_api(origin_dict): 
  # API Key 설정
  REST_API_KEY = "8830c13b6c30a909ce28c7def65bcf96"

  # origin_dict을 기준으로 모든 유니크한 정류장 쌍 생성
  # station_pairs = list(itertools.combinations(origin_dict.items(), 2))

  # 결과 저장 리스트
  results = []

  for station1, coord1 in origin_dict.items():


    exclude_key = station1 
    new_dict = {k: v for k, v in origin_dict.items() if k != exclude_key}
    for station2, coord2 in new_dict.items(): 
      print(station1, coord1 ,station2, coord2)
      # 호출
      result = get_directions(REST_API_KEY, coord1, coord2)

      if result and result.get('routes'):
          route = result['routes'][0]
          if route['result_code'] == 104: 
              distance_ = distance.distance((coord1[1], coord1[0]), (coord2[1], coord2[0])).m
              results.append({
                  '정류장1': station1,
                  '정류장2': station2,
                  '거리(m)': distance_
              })
              print(f"{station1} ↔ {station2}: {distance_}m")
          else:
              distance_ = route['summary']['distance']  # meter 단위
              results.append({
                  '정류장1': station1,
                  '정류장2': station2,
                  '거리(m)': distance_
              })
              print(f"{station1} ↔ {station2}: {distance_}m")
      else:
          print(f"길찾기 실패: {station1} ↔ {station2}")

      time.sleep(0.3)  # 너무 빠른 연속 호출 방지
  return pd.DataFrame(results)
  # 결과를 데이터프레임으로 저장


In [7]:
import pandas as pd 

상행_거리매트릭스 = use_api(origin_dict= origin_dict)
하행_거리매트릭스 = use_api(origin_dict= dest_dict)

00_오이도차고지 (126.69625, 37.3416) 02_오이도해양단지.옥터초교입구 (126.6932833, 37.3479833)
00_오이도차고지 ↔ 02_오이도해양단지.옥터초교입구: 920m
00_오이도차고지 (126.69625, 37.3416) 03_오이도중앙로입구 (126.6906833, 37.3477667)
00_오이도차고지 ↔ 03_오이도중앙로입구: 1197m
00_오이도차고지 (126.69625, 37.3416) 04_오이도종합어시장 (126.6896667, 37.345)
00_오이도차고지 ↔ 04_오이도종합어시장: 1498m
00_오이도차고지 (126.69625, 37.3416) 05_함상전망대 (126.6885167, 37.3418167)
00_오이도차고지 ↔ 05_함상전망대: 1864m
00_오이도차고지 (126.69625, 37.3416) 06_오이도박물관 (126.6916, 37.33605)
00_오이도차고지 ↔ 06_오이도박물관: 2453m
00_오이도차고지 (126.69625, 37.3416) 07_대부도입구 (126.6939667, 37.3353333)
00_오이도차고지 ↔ 07_대부도입구: 1056m
00_오이도차고지 (126.69625, 37.3416) 08_시화환경사업소 (126.6976667, 37.3371833)
00_오이도차고지 ↔ 08_시화환경사업소: 674m
00_오이도차고지 (126.69625, 37.3416) 09_시화염색단지입구 (126.69935, 37.3361667)
00_오이도차고지 ↔ 09_시화염색단지입구: 861m
00_오이도차고지 (126.69625, 37.3416) 10_삼양사 (126.7026833, 37.3341333)
00_오이도차고지 ↔ 10_삼양사: 1264m
00_오이도차고지 (126.69625, 37.3416) 11_열병합발전소 (126.7068167, 37.33155)
00_오이도차고지 ↔ 11_열병합발전소: 1699m
00_오이도차고지 (126.69625, 37.3416) 12_우진

KeyboardInterrupt: 

In [ ]:
# 위처럼 진행되면 성공 
# 보여주려고 한거라서 중간에 중단해서 에러난 것임 